Short notebook for demonstrating how scores can be computed from field simulations

In [1]:
import sys

In [2]:
sys.path.insert(0,'/KIMEL/tigrlab/projects/jjeyachandra/gmsh-sdk/lib/')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/miniconda2/envs/simnibs_env/lib/python2.7/site-packages')
sys.path.insert(0,'/home/jjeyachandra/simnibs_2.1.2/Python_modules/src')

In [17]:
import io, os, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import numpy as np

In [4]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        
        
        path = os.path.join(self.path,fullname) + '.ipynb'

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [14]:
#Load up the geolibrary as a module
glib_path = '/projects/jjeyachandra/rtms_optimize/notebooks/python2/'
nbload = NotebookLoader(glib_path)
geolib = nbload.load_module('geolib')

importing Jupyter notebook from /projects/jjeyachandra/rtms_optimize/notebooks/python2/geolib.ipynb


In [32]:
#Set pathing to simulation output, fem model, and partial parcel weights
field_path = '../../data/simnibs_output/test_simulation/sub-CMH090_TMS_1-0001_Magstim_70mm_Fig8_nii_scalar.msh'
fem_path = '../../data/simnibs_output/sub-CMH090.msh'
parcelmap_path = '../../output/tetra_parcels'

In [38]:
#Set up parcel data from MS-HBM parcellation model
num_parcels = 20
target_parcel = 2

In [39]:
#Load partial parcel map
proj_map = np.fromfile(parcelmap_path)
proj_map = proj_map.reshape( (proj_map.shape[0]//(num_parcels + 1), num_parcels + 1) )

In [40]:
#Load element data
_, elem_ids, _ = geolib.load_gmsh_elems(fem_path,entity=(3,2))

In [41]:
#Grab subset from field
norm_E = geolib.get_field_subset(field_path,elem_ids)

In [66]:
#Compute score
score = geolib.compute_field_score(norm_E,proj_map,target_parcel)

In [67]:
score

7401.630695451312